<a href="https://colab.research.google.com/github/MrFranciscoPaz/restmex/blob/main/seqtoseqmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import nltk

import gensim
import numpy as np
from nltk.corpus import stopwords
import pickle
import re
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
folder = './drive/MyDrive/data/'
namefiles = os.listdir(folder)

In [10]:
keywords = ["covid"]
spanishSW = stopwords.words('spanish')


In [9]:
def PrepararOraciones(str_Oracion):
  str_Oracion = str_Oracion.lower().strip()

  # Se crea un espacio entre las palabras y signos de puntuación
  str_Oracion = re.sub(r"([?.!,¿])", r" \1 ", str_Oracion)
  str_Oracion = re.sub(r'[" "]+', " ", str_Oracion)

  # Se reemplaza todo por un espacio excepto letras y signos especificados
  str_Oracion = re.sub(r"[^a-zA-Z?.!,¿]+", " ", str_Oracion)

  str_Oracion = str_Oracion.strip()

  # Se agrega el token de inicio y fin
  str_Oracion = '<start> ' + str_Oracion + ' <end>'

  with open(folder + file, encoding = "UTF-8") as f:
        lines = f.readlines()
  news = ' '.join(lines)
  news = news.split("#END")
  news = [x for x in news for y in keywords if y in x]
  news = ' '.join(lines)
  news = news.replace("#", "").replace(".", "").replace(",", "")
  news = news.replace(";", "").replace("(", "").replace(")", "")
  news = news.replace(":", "").replace("@", "").replace("|", "")
  news = news.split(" ")
  #words = [re.sub(r'"!\"$%&\(\)\*\+.,-/:;=?@\[\\\]^_`{|}~', "", x) for x in words]
  
  proc = [x.lower() for x in news if x not in sw]
  news = ' '.join(proc)
  return str_Oracion

In [ ]:
def Tokenizar(tupl_Frases):
  
  Tokenizador_Frases = tf.keras.preprocessing.text.Tokenizer(filters='')
  Tokenizador_Frases.fit_on_texts(tupl_Frases)

  tensor = Tokenizador_Frases.texts_to_sequences(tupl_Frases)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

  return tensor, Tokenizador_Frases

In [ ]:
def CargarDataset(str_RutaArchivo, nbr_CantEjemplos=None):
  
  tupl_Target, tupl_Input = PrepararDataset(str_RutaArchivo, nbr_CantEjemplos)

  # Idioma Input
  tensor_Input, Tokenizador_Frases_Input = Tokenizar(tupl_Input)
  
  # Idioma Target
  tensor_Target, Tokenizador_Frases_Target = Tokenizar(tupl_Target)

  return tensor_Input, tensor_Target, Tokenizador_Frases_Input, Tokenizador_Frases_Target

In [ ]:
nbr_CantEjemplos = 100
# nbr_CantEjemplos = 118964 

tensor_Input, tensor_Target, Tokenizador_Frases_Input, Tokenizador_Frases_Target = CargarDataset(str_RutaArchivo, nbr_CantEjemplos)

# Obtenemos el tamanio máximo de los tensores
nbr_TamanioMax_FraseTarget, nbr_TamanioMax_FraseInput = tensor_Target.shape[1], tensor_Input.shape[1]

In [ ]:
# 1. Quitar acentos
# 2. Limpiar oraciones
# 3. Regresar parejas de textos en el formtato: [Target, Input]
def PrepararDataset(str_RutaArchivo, nbr_CantEjemplos):

  str_Lineas = io.open(str_RutaArchivo, encoding='UTF-8').read().strip().split('\n')

  list_ParejaDeFrases = [[PrepararOraciones(str_Oracion) for str_Oracion in str_Linea.split('\t')]  for str_Linea in str_Lineas[:nbr_CantEjemplos]]
  
  return zip(*list_ParejaDeFrases)